# GPGN268 - Geophysical Data Analysis
## Data Story 03 - Seismology

**Student:** Lexi Herr
**Collaborators:** Jackson Kreiger, Mia Jungman
**Date:** April 6, 2023


In [1]:
# Import libraries
import os
import glob
import numpy as np
import matplotlib.pyplot as plt
import h5py
import pandas as pd

In [12]:
files_list = sorted(glob.glob('../data/DAS/Global_DAS_1078DT_25PR_14GL_5DEC_2023-02*.h5'))
data = h5py.File(files_list[0], 'r')

## Task 1 - Reading the data 

In [7]:
def strain_and_timing(path):
    """Return strain and timing
    Args: path(str): path to a given DAS .h5 file
    Returns: returns the strain and timing data in the file"""
    with h5py.File(path) as data:
        strain = data['Acquisition']['Raw[0]']['RawData'][:]
        time = data['Acquisition']['Raw[0]']['RawDataTime'][:]
    return strain, time

## Task 2 - Time and time again
### Task 2.1

In [10]:
def find_time(path):
    with h5py.File(path) as data:
        time_string = os.path.basename(data.filename).split('_')[-1].split('.')[0][:-1]
    return time_string

ts = find_time('../data/DAS/Global_DAS_1078DT_25PR_14GL_5DEC_2023-02-17T164133Z.h5')

### Task 2.2

In [5]:
time_stamp = pd.to_datetime(ts, format='%Y-%m-%dT%H%M%S', utc=True)

### Task 2.3

In [13]:
total_measurements = data['Acquisition']['Raw[0]']['RawDataTime'][:]
total_time = len(total_measurements) / 500
interval = 1/500

### Task 2.4

In [ ]:
end_time = time_stamp + pd.Timedelta(60, 's')

### Task 2.5

In [ ]:
time_utc = pd.date_range(start=time_stamp, end=end_time, periods=len(total_measurements))

### Task 2.6

In [ ]:
time = time_utc.tz_convert('US/Mountain')

### Task 2.7

In [11]:
def data_to_mountain_time(path):
    """Return time values converted to mountain time
    Args: path(str): path to a given DAS .h5 file
    Returns: MST of time data in the file"""
    with h5py.File(path) as data:
        time_string = os.path.basename(data.filename).split('_')[-1].split('.')[0][:-1]
        ts = find_time(path)
        time_stamp = pd.to_datetime(ts, format='%Y-%m-%dT%H%M%S', utc=True)
        total_measurements = data['Acquisition']['Raw[0]']['RawDataTime'][:]
        end_time = time_stamp + pd.Timedelta(60, 's')
        time_utc = pd.date_range(start=time_stamp, end=end_time, periods=len(total_measurements))
        time = time_utc.tz_convert('US/Mountain')
    return time

## Task 3

In [ ]:
def strain_rate(path):
    with h5py.File(path) as data:
        strain, time = strain_and_timing(path)
        strain_diff = np.diff(strain, axis=0)
        dt = (time[1]-time[0]).total_seconds()
        sr = strain_diff/dt
        time = time[:-1]
    return sr

## Task 4

In [9]:
def strain_rate_and_mountain_time(path):
    """Return strain rate and mountain time
    Args: path(str): path to a given DAS .h5 file
    Returns: returns the strain rate and MST of time data in file"""
    mountain_time = data_to_mountain_time(path)
    strain, time = strain_and_timing(path)
    strain_diff = np.diff(strain,axis=0)
    mountain_time = mountain_time[:-1]
    dt = (mountain_time[1]-mountain_time[0]).total_seconds()
    strain_rate = strain_diff/dt
    return strain_rate, mountain_time 